## MEG Preprocessing

Details

In [38]:
import matplotlib.pyplot as plt
import mne
import numpy as np
import os
import pandas as pd
import pickle

from pathlib import Path

### Specify subject ID and file locations

In [127]:
subject = 4
tmin = -0.1
tmax = 0.5

save_folder = Path(rf'F:\Data\THINGS\MEG_raw\preprocessed')
meg_folder = Path('D:\Data_Holding')
samples_attributes_folder = Path('F:\Data\THINGS\misc')

#### Data Splits (Train/Val/Test)

* Each category folder contains 12 images
* An earlier analysis organised the following
* Dev and Test contain one example from each
* Training data is thus 11 images per category
* The lists of the filesname are saved in `.pkl` files and loaded below

In [3]:
train_imgs = pickle.load(open('train_imgs_list.pkl', 'rb'))
train_imgs_list = [x[0].split('/')[-1] for x in train_imgs]

val_imgs = pickle.load(open('val_imgs_list.pkl', 'rb'))
val_imgs_list = [x[0].split('/')[-1] for x in val_imgs]

test_imgs = pickle.load(open('test_imgs_list.pkl', 'rb'))
test_imgs_list = [x[0].split('/')[-1] for x in test_imgs]

The following functions help to assign each image entry to a specific `train` / `val` or `test` split

In [4]:
def get_filename(image_path):
    return image_path.split('/')[-1]

def get_split(row):
    if row['filename'] in val_imgs_list:
        return 'val'
    elif row['trial_type'] == "test":
        return 'test'
    else:
        return 'train'

The main preprocessing function is given below.

* It takes in a subject ID, a session and a run number.
* In the folder `sub-MIGBEG1`, the main folder for subject 1, there are 12 session folders
* Inside each session folder is a list of the separate experiment runs for that sesson

In [131]:
def preprocess_meg(sub_no, session_no, run_no, downsample_sfreq=200):

    m2_folder = Path(meg_folder  / f'sub-BIGMEG{sub_no}')
    m2_events = pd.read_csv(Path(samples_attributes_folder / f'sample_attributes_P{sub_no}.csv'))
    m2_events_curr = m2_events.apply(lambda row: row[m2_events['session_nr'] == session_no])
    m2_events_curr = m2_events_curr.apply(lambda row: row[m2_events_curr['run_nr'] == run_no])
    session_folders = list(m2_folder.iterdir())[0]
    m2_folder_sess = Path(m2_folder, f'ses-{session_no:02d}/meg')
    print(m2_folder_sess)
    
    try:
        curr_run_path = list(m2_folder_sess.glob(f'./*run-{run_no:02d}*ds'))[0]
    except:
        print(f"Didn't get any results for run number: {run_no}")
        raise(Exception("STOP"))
    curr_run = mne.io.read_raw_ctf(curr_run_path, preload=True)

    curr_run.filter(l_freq=0.1, h_freq=40., h_trans_bandwidth=0.5, filter_length='10s', phase='zero-double', fir_design='firwin2');

    events = mne.find_events(curr_run, stim_channel='UPPT001')
    events = events[:-1]
    assert len(events) == len(m2_events_curr)
    events[:, 2] = np.array(m2_events_curr['category_nr'])

    epochs = mne.Epochs(curr_run, events, tmin=tmin, tmax=tmax, picks=['meg'], baseline=(-0.1, 0), proj=True)
    epochs.metadata = m2_events_curr

    epochs.metadata['filename'] = epochs.metadata['image_path'].apply(get_filename)
    epochs.metadata['split'] = epochs.metadata.apply(get_split, axis=1)
    
    epochs.load_data()
    epochs = epochs.resample(sfreq=downsample_sfreq)
    print(epochs.get_data().shape)
    
    save_file = f'sess-{session_no:02d}_run-{run_no:02d}-epo.fif'
    print(f'Saving: {save_folder / save_file} ')
    epochs.save(save_folder / save_file, overwrite=True)

Run the code for all subjects, sessions and runs

In [132]:
# 12 sessions per subject
# 10 runs per subject

for sess in range(1,13):
    for run in range(1,11):
        preprocess_meg(subject, sess, run, downsample_sfreq=200)

D:\Data_Holding\sub-BIGMEG4\ses-01\meg
ds directory : D:\Data_Holding\sub-BIGMEG4\ses-01\meg\sub-BIGMEG4_ses-01_task-main_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -1.37   72.35    0.00 mm <->   -1.37   72.35    0.00 mm (orig :  -38.97   62.23 -251.85 mm) diff =    0.000 mm
       1.37  -72.35    0.00 mm <->    1.37  -72.35    0.00 mm (orig :   61.59  -41.85 -252.94 mm) diff =    0.000 mm
      85.33    0.00    0.00 mm <->   85.33    0.00    0.00 mm (orig :   73.48   67.91 -243.13 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for D:\Data_Holding\sub-BIGMEG4\ses-01\meg\sub-BIGMEG4_ses-01_task-main_run-01_meg.ds\sub-BIGMEG4_ses-01_task-main_run-01_meg.meg4: 
    System clock channel is available, checking which samples ar

C:\Users\murph\AppData\Local\Temp\ipykernel_24308\1291071959.py:18: RuntimeWarning: Attenuation at stop frequency 0.00 Hz is only 14.21 dB. Increase filter_length for higher attenuation.
  curr_run.filter(l_freq=0.1, h_freq=40., h_trans_bandwidth=0.5, filter_length='10s', phase='zero-double', fir_design='firwin2');
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Trigger channel UPPT001 has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
227 events found on stim channel UPPT001
Event IDs: [32 64]
Not setting metadata
226 matching events found


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    6.7s finished


Applying baseline correction (mode: mean)
0 projection items activated
Adding metadata with 18 columns
Using data from preloaded Raw for 226 events and 721 original time points ...
0 bad epochs dropped
(226, 300, 120)
Saving: F:\Data\THINGS\MEG_raw\preprocessed\sess-01_run-01-epo.fif 
Overwriting existing file.
Overwriting existing file.


...

C:\Users\murph\AppData\Local\Temp\ipykernel_24308\1291071959.py:18: RuntimeWarning: Attenuation at stop frequency 0.00 Hz is only 14.21 dB. Increase filter_length for higher attenuation.
  curr_run.filter(l_freq=0.1, h_freq=40., h_trans_bandwidth=0.5, filter_length='10s', phase='zero-double', fir_design='firwin2');
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Trigger channel UPPT001 has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
226 events found on stim channel UPPT001
Event IDs: [32 64]
Not setting metadata
225 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Adding metadata with 18 columns


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    6.5s finished


Using data from preloaded Raw for 225 events and 721 original time points ...
0 bad epochs dropped
(225, 300, 120)
Saving: F:\Data\THINGS\MEG_raw\preprocessed\sess-12_run-10-epo.fif 


C:\Users\murph\AppData\Local\Temp\ipykernel_24308\1291071959.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(epochs.get_data().shape)


Find which index is MRO11 and mark it for removal

In [85]:
subj_folder = Path('F:\Data\THINGS\MEG_raw\preprocessed\sub1')
files = os.listdir(subj_folder)
file = subj_folder / files[5]
tmp = mne.read_epochs(subj_folder / file)

occipital_indices = [i for i, x in enumerate(tmp.info['ch_names'])]
mro11_idx = [i for i, x in enumerate(tmp.info['ch_names']) if 'MRO11' in x][0]
print(mro11_idx)
occipital_indices = [x for x in occipital_indices if x != mro11_idx]
print(occipital_indices)

occipital_indices = np.array([84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,296,297,298])

Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-04-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
215
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,

In [94]:
test = np.zeros((1854, 300 ,11))
test[:, occipital_indices, :].shape

(1854, 39, 11)

In [79]:
tmp.info['nchan']

300

In [86]:
len(occipital_indices)

39

In [147]:
def extract_meg_window(sub_no, data_folder, tmin, tmax, sensors=None):
    """ Given a folder of preprocessed MEG data, extract and concatenate a specific time window for
        the training, validation and test data.
        Args:
        data_folder: where the preprocessed sessions are stored

        sensors = None: a list of indices in the channels dimension to extract
        """
    
    files = list(data_folder.glob('*.fif'))
    print(f'Found {len(files)} files')

    train = []
    val = []
    test = []

    dev_head_t = None 

    for file in files[:]:

        tmp = mne.read_epochs(subj_folder / file)
        tmp.baseline = None
        tmp.crop(tmin=tmin, tmax=tmax)

        #print(tmp.ch_names)

        if dev_head_t is None:
            dev_head_t = tmp.info['dev_head_t']

        tmp.info['dev_head_t'] = dev_head_t # https://github.com/mne-tools/mne-python/issues/2109

        tmp_train = tmp['  split == "train"    ']
        tmp_val = tmp['  split == "val"    ']
        tmp_test = tmp['  split == "test"    ']

        train.append(tmp_train)
        val.append(tmp_val)
        test.append(tmp_test)
        
    #train_epochs = mne.concatenate_epochs(train)
    val_epochs = mne.concatenate_epochs(val)
    #test_epochs = mne.concatenate_epochs(test)

    # train_data = train_epochs['trial_type != "catch"'].get_data()
    # train_data = train_data[:, sensors, :]
    # print(f'{train_data.shape =}')
    # train_data = np.squeeze(np.mean(train_data, axis=2))
    # print(f'{train_data.shape = }')
    # fname = save_folder / f'sub{sub_no}' / f'meg_sub{sub_no}_train_{int(tmin*1000)}_{int(tmax*1000)}.npy'
    # np.save(open(fname,'wb'), train_data)

    val_data = val_epochs.get_data()
    print(f'{val_data.shape = }')
    #assert val_data.shape == (1854, 300, 11), f'{val_data.shape = }'
    val_data = val_data[:, sensors, :]
    #assert val_data.shape == (1854, 39, 11), f"{val_data.shape = }"
    print(f'{val_data.shape = }')
    #val_data = np.mean(val_data, axis=2)
    #assert val_data.shape == (1854, 39), f"{val_data.shape = }"

    # Flatten here for Figure 2 CORnet-S increasing dimensionality analysis
    val_data = np.reshape(val_data, (val_data.shape[0], -1))

    print(f'{val_data.shape = }')
    assert val_data.shape[0] == 1854, "Val data incorrect shape"
    np.save(open(save_folder / f'sub{sub_no}' /  f'meg_sub{sub_no}_val_{int(tmin*1000)}_{int(tmax*1000)}_noavg.npy','wb'), val_data)

    #n_test_images = len(np.unique(test_epochs.metadata['filename']))
    #nchan = len(sensors) if sensors is not None else test_epochs.info['nchan']
    #y_test = np.zeros((n_test_images, nchan))

    for i, test_stim in enumerate(test_imgs):

        continue
        
        tmp_filename, tmp_idx = test_stim
        tmp_filename = tmp_filename.split('/')[-1]
        tmp_epochs = test_epochs['filename == "{}"'.format(tmp_filename)]
        tmp_epochs.load_data()
        assert len(tmp_epochs) == 12, f"Found only {len(tmp_epochs)} events (expected 12)"
        tmp_evoked = tmp_epochs.average()
        tmp_evoked = tmp_evoked.data
        print(f'{tmp_evoked.ndim = }')
        assert tmp_evoked.ndim == 3, "ndim unexpected"
        tmp_evoked = tmp_evoked[:, sensors, :]
        print(f'{tmp_evoked.ndim = }')
        tmp_evoked = np.mean(tmp_evoked, axis=2)
        print(f'{tmp_evoked.ndim = }')
        tmp_evoked = np.squeeze(tmp_evoked)
        print(f'{tmp_evoked.ndim = }')
        y_test[i,:] = tmp_evoked

    #np.save(open(save_folder / f'sub{sub_no}' / f'meg_sub{sub_no}_test_{int(tmin*1000)}_{int(tmax*1000)}.npy','wb'), y_test)
    #return val_epochs, val_data

In [148]:



#extract_meg_window(sub_no=3, data_folder=data_folder,  tmin=tmin, tmax=tmax, sensors=occipital_indices)
#val_epochs, val_data = extract_meg_window(sub_no=subject, data_folder=data_folder,  tmin=tmin, tmax=tmax, sensors=occipital_indices)

sub_no = 1
data_folder = Path(f'F:\Data\THINGS\MEG_raw\preprocessed\sub{sub_no}')
extract_meg_window(sub_no, data_folder, 0.05, 0.10, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.15, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.20, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.25, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.30, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.35, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.40, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.45, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.50, sensors=occipital_indices)

sub_no = 2
data_folder = Path(f'F:\Data\THINGS\MEG_raw\preprocessed\sub{sub_no}')
extract_meg_window(sub_no, data_folder, 0.05, 0.10, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.15, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.20, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.25, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.30, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.35, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.40, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.45, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.50, sensors=occipital_indices)

sub_no = 4
data_folder = Path(f'F:\Data\THINGS\MEG_raw\preprocessed\sub{sub_no}')
extract_meg_window(sub_no, data_folder, 0.05, 0.10, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.15, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.20, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.25, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.30, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.35, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.40, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.45, sensors=occipital_indices)
extract_meg_window(sub_no, data_folder, 0.05, 0.50, sensors=occipital_indices)

Found 120 files
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-01-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-02-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-03-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No bas

C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  val_data = val_epochs.get_data()


Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-02-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-03-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-04-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction

C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  val_data = val_epochs.get_data()


    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-03-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-04-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
226 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Data\THINGS\MEG_raw\preprocessed\sub1\sess-01_run-05-epo.fif

...

C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:23: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.495 s)
  tmp.crop(tmin=tmin, tmax=tmax)
C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:23: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.495 s)
  tmp.crop(tmin=tmin, tmax=tmax)


Reading F:\Data\THINGS\MEG_raw\preprocessed\sub4\sess-12_run-10-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -100.00 ...     495.00 ms
        5 CTF compensation matrices available
Adding metadata with 20 columns
225 matching events found
No baseline correction applied
0 projection items activated


C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:23: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.495 s)
  tmp.crop(tmin=tmin, tmax=tmax)


Adding metadata with 20 columns
1854 matching events found
No baseline correction applied
val_data.shape = (1854, 300, 90)
val_data.shape = (1854, 39, 90)
val_data.shape = (1854, 3510)


C:\Users\murph\AppData\Local\Temp\ipykernel_24308\547805743.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  val_data = val_epochs.get_data()
